In [1]:
# Generates the test data (splits the loading in transcript part with the actual iteration over exon part)
"""
tests (not 1 exon, gene level 1)
exact transcript to cDNA mapping

test_stricter_human_genome/ (not 1 exon, gene level 1, transcript level 1, transcript support level 1)


test_stricter_human_genome_with_slight_extension (not just 1 exon, gene level 1, transcript level 1, transcript support level 1, reference sequence doesn't have masked regions (hard masked NNN or soft masked lowercase letters))

extend by 2 nt on either side of reference

test_stricter_human_genome_with_1_nt_ext (not just 1 exon, gene level 1, transcript level 1, transcript support level 1, reference sequence doesn't have masked regions (hard masked NNN or soft masked lowercase letters))

extend reference by 1nt only (to check if pairagon still works)
"""



import gffutils
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import os
import subprocess

def reverse_complement(dna:str):
  ans = ""
  nucleotide_map = {'A':'T','C':'G','G':'C','T':'A'}
  for c in dna[-1::-1]:
    ans+=nucleotide_map[c]
  return ans
    
    

def make_path(path):
  if not os.path.exists(path):
    os.makedirs(path)

annotation_file_name = "mouse/gencode.vM34.annotation.gff3"
transcript_file_name = "mouse/gencode.vM34.transcripts.fa"
genome_file_name = "mouse/GRCm39.primary_assembly.genome.fa"

# db = gffutils.create_db(annotation_file_name, dbfn='mouse.db', force=True, keep_order=False,
# merge_strategy='merge', sort_attribute_values=False)

db = gffutils.FeatureDB('./mouse/mouse.db', keep_order=True)
print("Loaded in db!")
transcript_sequences = SeqIO.parse(open(transcript_file_name),'fasta')
transcript_id_to_sequence = {seq_record.id.split("|")[0]:seq_record.seq for seq_record in SeqIO.parse(transcript_file_name,"fasta") }
chromosome_id_to_sequence = {seq_record.id.split(" ")[0]:seq_record.seq for seq_record in SeqIO.parse(genome_file_name,"fasta") }
print("Loaded in transcript sequences!")


Loaded in db!
Loaded in transcript sequences!


In [3]:
import random

sub_dir = "mouse_mutated_sub"
# for reference genome
left_offset = 2
right_offset = 2

cnt = 0

def mutatecDNA(cDNA:str):
  mutation_rate = 0.0042
  to_sub = {'A':['G','G','C','T'],'T':['G','G','A','C'],'C':['G','G','A','T'],'G':['T']}
  mutated_cdna = ""
  for char in cDNA:
    if random.uniform(0,1)<=mutation_rate:
      mutated_cdna+=random.choice(to_sub[char])
    else:
      mutated_cdna+=char
  return mutated_cdna

for gene in db.features_of_type('gene',order_by='start'):
  if gene['level'][0] != "1":
    continue
  
  for transcript in db.children(gene,featuretype='transcript',order_by='start'):
    transcript_id = transcript.id
    cur_chromsome = chromosome_id_to_sequence[transcript.seqid]
    if transcript.start is None or transcript.end is None:
      continue
    if transcript_id in transcript_id_to_sequence:
      
      fakeCDNA = "" # we'll just make this as reference to compare against actual transcript... just in case (but we'll still use the actual transcript)
      
      
      reference_cDNA = transcript_id_to_sequence[transcript_id]
      if transcript.start-left_offset<1 or transcript.end+right_offset>len(cur_chromsome):
        print("Transcript starts too early! Pairagon will be upset")
        continue
      reference_sequence = cur_chromsome[transcript.start-1-left_offset:transcript.end+right_offset]
      exon_count = 0
      exon_intervals = []
      # if not transcript['level'][0]=="1":
      # print(transcript['levefl'][0] != "1" , transcript['transcript_support_level'][0]!="1")
      if 'level' not in transcript.attributes or transcript['level'][0] != "1" or 'transcript_support_level' not in transcript.attributes or transcript['transcript_support_level'][0]!="1":
        continue
      # if 'MANE_Select' not in transcript.attributes:
      #   continue
      
      # if 'ccdsid' not in transcript.attributes or len(transcript['ccdsid']) == 0:
      #   # print("skipping non-coding transcript")
      #   continue
      if not set(reference_sequence) <= set("ACTG"):
          print("Masked region detected, skipping",transcript_id,reference_sequence)
          # no N's or lowercase letters allowed (basically we don't want soft-masking/uncertain data)
          continue
      
      for exon in db.children(transcript,featuretype='exon',order_by='start'):
        assert(exon['level'][0]=="1")
        assert(exon["transcript_support_level"][0]=="1")
        
        assert(exon.start is not None and exon.end is not None)
        startI = exon.start-transcript.start
        endI = exon.end-transcript.start
        fakeCDNA+=reference_sequence[startI+left_offset:endI+1+left_offset] #also offset here. this is a bit annoying
        exon_count+=1
        exon_intervals.append(f"{startI+1+left_offset}/{endI+1+left_offset}") # return it back to 1-indexing (and then add the 2-nt offset at the start)
      if exon_count <= 1:
        print("1 exon",transcript_id)
        assert(exon_count!= 0)
        continue # really boring transcript
      # print(fakeCDNA)
      # print(reference_cDNA)
      assert(fakeCDNA==reference_cDNA or fakeCDNA==reverse_complement(reference_cDNA)) 
      
      exon_intervals.append("+" if fakeCDNA==reference_cDNA else "-") # directionality (template vs. non-template strand)
      reference_cDNA = mutatecDNA(reference_cDNA) # da only substitution mutation
      assert(gene.id != None)
      make_path(f"{sub_dir}/{transcript_id}")
      SeqIO.write(SeqRecord(reference_sequence,transcript_id,description=gene.id),tfn:=f"{sub_dir}/{transcript_id}/genome.fa","fasta")
      SeqIO.write(SeqRecord(reference_cDNA,f"cDNA_{transcript_id}",description=gene.id),cdna_fn:=f"{sub_dir}/{transcript_id}/cdna.fa","fasta") # type: ignore
      open(f"{sub_dir}/{transcript_id}/ans.txt", "w").write("\n".join(exon_intervals))
      cnt += 1
      
      # subprocess.run(f"/Users/work/Documents/SD_Projects/gmap-2024/bin/gmap {cdna_fn} -g {tfn} -E genomic -f 3 > tests/{transcript_id}/result.gmap",check=True,shell=True,stdout=subprocess.DEVNULL,stderr=subprocess.DEVNULL)
    else:
      print(f"Skipping {transcript_id} cuz not found")

print(cnt)
# This time filtering out everything where genome is masked
# Filter out non-coding regions

/home/work/cb-proj/path/to/venv/lib/python3.11/site-packages/Bio/SeqRecord.py:229: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


583
